# 3 Extracting Volatility from the Orderbook

There is no analytical solution to isolate $\sigma$. Therefore, generally most practictioners use numerical methods to solve for these values.

Some of the possible methods are
1. Newton-Raphson
2. Bisection Method

Questions:
- What does the market tell us based on the option prices?
- Are the predicted values for the options close to the market prices?
- Is there an opportunity where the market is mispricing the option?
- What is the right volatility model -> Is the market correct right now?

Notes
- The Volatility Surface
  - Chapter 1: 
    $$v_L = \frac{\frac{\partial w}{\partial T}}{1 - \frac{y}{w} \frac{\partial \omega}{\partial y} + \frac{1}{4}(-\frac{1}{4} - \frac{1}{w} + \frac{y^2}{w^2})(\frac{\partial w}{\partial y})^2 + \frac{1}{2} \frac{\partial^2 w}{\partial y^2}}$$
    - black scholes implied total variance - w
    - log-strike y

    $$\sigma^2 (K,T,S_0) = E [v_t | S_T = K]$$
    - That is, local variance is the risk-neutral expectation of the instantaneous variance conditional on the final stock price ST being equal to the strike price K.
    - 

In [1]:
import sys
import os

root_dir = os.path.abspath("/Users/ryant/Documents/Github/digital-asset-options-pricing")
if root_dir not in sys.path:
    sys.path.append(root_dir)
os.chdir(root_dir)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pytz

In [2]:
df_options_btc = pd.read_parquet('/Users/ryant/Documents/Github/digital-asset-options-pricing/src/data/deribit/data/deribit_btc_snapshot_011124_301124.parquet')
df_options_btc

,date,open_interest,underlyer,expiration_date,claim_type,mark_price,mark_iv,bid_iv,ask_iv,exercise,settlement,strike,best_bid_price,best_ask_price,underlyer_spot,forward_price,volume,best_ask_amount,best_bid_amount,instrument_name
0,1730419200000,107.7,BTC_USD,2024-11-08T08:00:00.000Z,put,0.0810,68.41,66.03,72.38,European,cash,75000,0.0800,0.0830,70190.90,70373.62,9.0,7.8,6.6,BTC-8NOV24-75000-P
1,1730419200000,2378.0,BTC_USD,2024-11-08T08:00:00.000Z,call,0.0154,68.41,68.68,69.74,European,cash,75000,0.0155,0.0160,70190.90,70373.62,265.5,33.5,0.1,BTC-8NOV24-75000-C
2,1730419200000,97.5,BTC_USD,2024-11-08T08:00:00.000Z,put,0.0704,68.11,65.16,72.07,European,cash,74000,0.0690,0.0725,70190.90,70373.62,25.0,17.6,25.0,BTC-8NOV24-74000-P
3,1730419200000,531.4,BTC_USD,2024-11-08T08:00:00.000Z,call,0.0190,68.11,67.22,69.19,European,cash,74000,0.0185,0.0195,70190.90,70373.62,148.8,23.0,58.6,BTC-8NOV24-74000-C
4,1730419200000,174.3,BTC_USD,2024-11-08T08:00:00.000Z,put,0.0606,68.05,66.87,69.67,European,cash,73000,0.0595,0.0615,70190.90,70373.62,23.7,26.6,26.1,BTC-8NOV24-73000-P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455853,1733010300000,124.2,BTC_USD,2025-09-26T08:00:00.000Z,call,0.2040,60.29,59.44,61.14,European,cash,110000,0.2010,0.2070,96514.72,106886.94,0.0,11.1,9.9,BTC-26SEP25-110000-C
2455854,1733010300000,2.1,BTC_USD,2025-09-26T08:00:00.000Z,put,0.2036,60.03,57.97,61.44,European,cash,105000,0.1965,0.2085,96514.72,106886.94,0.0,19.4,12.4,BTC-26SEP25-105000-P
2455855,1733010300000,3.4,BTC_USD,2025-09-26T08:00:00.000Z,call,0.2213,60.03,58.93,60.96,European,cash,105000,0.2175,0.2245,96514.72,106886.94,0.1,1.3,1.8,BTC-26SEP25-105000-C
2455856,1733010300000,28.7,BTC_USD,2025-09-26T08:00:00.000Z,put,0.1759,59.81,58.94,60.74,European,cash,100000,0.1730,0.1790,96514.72,106886.94,0.0,1.2,12.5,BTC-26SEP25-100000-P


In [4]:
df_iv = df_options_btc[['date', 'instrument_name', 'expiration_date', 'strike', 'claim_type', 'bid_iv', 'mark_iv']].copy()
df_iv

,date,instrument_name,expiration_date,strike,claim_type,bid_iv,mark_iv
0,1730419200000,BTC-8NOV24-75000-P,2024-11-08T08:00:00.000Z,75000,put,66.03,68.41
1,1730419200000,BTC-8NOV24-75000-C,2024-11-08T08:00:00.000Z,75000,call,68.68,68.41
2,1730419200000,BTC-8NOV24-74000-P,2024-11-08T08:00:00.000Z,74000,put,65.16,68.11
3,1730419200000,BTC-8NOV24-74000-C,2024-11-08T08:00:00.000Z,74000,call,67.22,68.11
4,1730419200000,BTC-8NOV24-73000-P,2024-11-08T08:00:00.000Z,73000,put,66.87,68.05
...,...,...,...,...,...,...,...
2455853,1733010300000,BTC-26SEP25-110000-C,2025-09-26T08:00:00.000Z,110000,call,59.44,60.29
2455854,1733010300000,BTC-26SEP25-105000-P,2025-09-26T08:00:00.000Z,105000,put,57.97,60.03
2455855,1733010300000,BTC-26SEP25-105000-C,2025-09-26T08:00:00.000Z,105000,call,58.93,60.03
2455856,1733010300000,BTC-26SEP25-100000-P,2025-09-26T08:00:00.000Z,100000,put,58.94,59.81


# Newton Ra